In [1]:
import tabula 
import pandas as pd
from tika import parser
import os
import copy
import urllib
import wget
import ssl

In [4]:
def download_pdfs(start, end):
    ssl._create_default_https_context = ssl._create_unverified_context
    url = 'https://e-eco.gov.ua/parlor/applications/viewfile/id/'
    dest = 'declarations/'
    for i in range(start, end):
        if i % 250 == 0:
            print(i)
        if not os.path.isfile(dest + str(i)):
            try:
                wget.download(url + str(i), out='dec/' + str(i))
            except urllib.error.HTTPError:
                pass

In [5]:
# download_pdfs(43000, 45000)

43000
43250
43500
43750
44000
44250
44500
44750


In [6]:
def parsing_table(file, df_all1, df_all2, df_all3, dai):  
#     if not os.path.isfile(file):
    try:
        raw = parser.from_file(file)
#     except urllib.error.HTTPError:
    except FileNotFoundError:
        skipped.append(dai)
        return df_all1, df_all2, df_all3
    if raw['status'] == 200 and raw['content'] is not None and len(raw['content'].strip().split()) > 200 and raw['content'].strip().split()[4] == '№' and raw['content'].strip().split()[5][0] == 'D':
        df = tabula.read_pdf(file, pages=1, spreadsheet=True, pandas_options={'header': None}, multiple_tables=True)
        if type(df) == list and len(df) == 2:
            df[0]['decl_num'] = raw['content'].strip().split()[5]
            df[1]['decl_num'] = raw['content'].strip().split()[5]
        else:
            print('akusi', dai)
        if df_all1 is None:
            df_all1 = df[0]
            df_all2 = df[1]
        else:
            df_all1 = pd.concat([df_all1, df[0]], ignore_index=True)
            df_all2 = pd.concat([df_all2, df[1]], ignore_index=True)
        df = tabula.read_pdf(file, pages=2, spreadsheet=True, pandas_options={'header': None}, multiple_tables=True)
        if type(df) == list and len(df) >= 1:
            df = [j for j in df if type(j) == pd.core.frame.DataFrame and j.shape != (1, 1)][0]
        for i in range(3, int(raw['metadata']['xmpTPg:NPages']) + 1):
            f2 = tabula.read_pdf(file,
                         pages=i, pandas_options={'header': None}, spreadsheet=True, multiple_tables=True)
            if type(f2) == list and len(f2) >= 1:
                f2 = [j for j in f2 if type(j) == pd.core.frame.DataFrame and j.shape != (1, 1)][0]
            if type(f2) != list:
                df = pd.concat([df, f2], ignore_index=True)
        df['decl_num'] = raw['content'].strip().split()[5]
        if df_all3 is None:
            df_all3 = df
        else:
            df_all3 = pd.concat([df_all3, df], ignore_index=True)
    return df_all1, df_all2, df_all3

In [7]:
def data_cleaning(dat1, dat2, dat3):
#     print(path_to, file_num, 2)
    dat3 = dat3.loc[(dat3[2].isnull() == False) & (dat3[4].isnull() == False) & (dat3[2] != '3' ) & (dat3[4] != 'Клас\rнебезпеки')]
    for i in range(12):
        dat3[i] = dat3[i].apply(lambda x: str(x).replace('\r', ' '))
    dat3.columns = ['No','trash_name','code','alter_name','class','storage','value',
                                 'value_prediction','destination','value_destination','operation','operation_desc', 'decl_num']

    dat1[0] = dat1[0].apply(lambda x: str(x).replace('\r', ' '))
    dat1[1] = dat1[1].apply(lambda x: str(x).replace('\r', ' '))
    
    print(dat2[dat2[0].isnull()][dat2[dat2[0].isnull()][5] != '(x1)'], "AAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
    for i in range(6):
        dat2[i] = dat2[i].apply(lambda x: str(x).replace('\r', ' '))
    dat2 = dat2.loc[(dat2[0] != 'Період утворення відходів') & (dat2[5] != '(x1)')]
    dat2.columns = ['waste_creation_time','index_gen_wastes','amount_I','amount_II', 'amount_III', 'amount_IV', 'decl_num']
    return dat1, dat2, dat3

In [8]:
def data_saving(datas, file_num):
    datas[0], datas[1], datas[2] = data_cleaning(datas[0], datas[1], datas[2])
    for j in range(3):
        datas[j].to_csv(path_to + files[j] + str(file_num) + ".csv", sep=';', index=False)
#     file_num += 1000
#         if not os.path.isfile(path_to + files[j]):
#             datas[j].to_csv(path_to + files[j], sep=';', index=False)
#         else:
#             with open(path_to + files[j], 'a') as f:
#                 datas[j].to_csv(path_to + files[j], sep=';', index=False, header=False)
    return [None, None, None]

In [9]:
datas = [None, None, None]
# path_from = 'https://e-eco.gov.ua/parlor/applications/viewfile/id/' # in case we download on flight
path_from = './declarations/' # where are downloaded declarations
path_to = "./dec/data/d2/" # where we save files
file_num = 41000 # file_num - 999 = starting file of our work => min file_num == 1000, and file_num % 1000 == 0
files = ["F", "S", "T"]
skipped = []
min_range = file_num - 999
max_range = 42001 # max range of files to 

for i in range(min_range, max_range):
    if i % 100 == 0:
        print('--', i)
    datas[0], datas[1], datas[2] = parsing_table(path_from + str(i), datas[0], datas[1], datas[2], i)
    if i % 1000 == 0:
        datas = data_saving(datas, file_num)
        file_num += 1000
        
datas = data_saving(datas, 0)

2019-05-02 14:29:59,911 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar to /tmp/tika-server.jar.
2019-05-02 14:30:51,503 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar.md5 to /tmp/tika-server.jar.md5.
2019-05-02 14:30:52,218 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


-- 40100
-- 40200
-- 40300
-- 40400
-- 40500
-- 40600
-- 40700
-- 40800
-- 40900
-- 41000


/home/roman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, decl_num]
Index: [] AAAAAAAAAAAAAAAAAAAAAAAAAAAAA
-- 41100
-- 41200
-- 41300
-- 41400
-- 41500
-- 41600
-- 41700
-- 41800
-- 41900
-- 42000


/home/roman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, decl_num]
Index: [] AAAAAAAAAAAAAAAAAAAAAAAAAAAAA


AttributeError: 'NoneType' object has no attribute 'loc'

In [11]:
print(pd.read_csv("./dec/data/fulls/Declarations_first_table.csv", sep=';').shape)
print(pd.read_csv("./dec/data/fulls/Declarations_second_table.csv", sep=';').shape)
print(pd.read_csv("./dec/data/fulls/Declarations_third_table.csv", sep=';').shape)

(108540, 3)
(43416, 7)
(189335, 13)


/home/roman/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
#join files into one csv file
for f in files:
    data1 = pd.read_csv("./dec/data/d2/" + f + "1000.csv", sep=';')
    for i in range(2000,43000,1000):
        data2 = pd.read_csv("./dec/data/d2/" + f + str(i) + ".csv", sep=';')
        data1 = pd.concat([data1,data2], ignore_index=True)
    if f == 'F':
        data1.to_csv("./dec/data/fulls/Declarations_first_table.csv", sep=';', index=False)
    elif f == 'S':
        data1.to_csv("./dec/data/fulls/Declarations_second_table.csv", sep=';', index=False)
    elif f == 'T':
        data1.to_csv("./dec/data/fulls/Declarations_third_table.csv", sep=';', index=False)

In [17]:
print(pd.read_csv("./dec/data/fulls/Declarations_first_table.csv", sep=';').shape)
print(pd.read_csv("./dec/data/fulls/Declarations_second_table.csv", sep=';').shape)
print(pd.read_csv("./dec/data/fulls/Declarations_third_table.csv", sep=';').shape)

(109630, 3)
(43852, 7)
(190915, 13)


In [18]:
pd.read_csv("./dec/data/fulls/Declarations_third_table.csv", sep=';')

,No,trash_name,code,alter_name,class,storage,value,value_prediction,destination,value_destination,operation,operation_desc,decl_num
0,1,"Масла та мастила моторні, трансмісійні нехлоро...",6000.2.8.09,Відпрацьовані мастила,ІІ,0,0.012,0.012,38563155 ТОВ «Науково- дослідний інститут «УКР...,0.012,R9,NaN,D1221855500_00001_260716
1,2,"Матеріали обтиральні зіпсовані, відпрацьовані ...",7730.3.1.06,Промащене ганчір’я,ІІІ,0,0.100,0.211,38563155 ТОВ «Науково- дослідний інститут «УКР...,0.100,R5,NaN,D1221855500_00001_260716
2,3,"Відходи комунальні (міські) змішані, у т. ч. с...",7720.3.1.01,Побутове сміття,ІV,0,18.000,84.18,38641065 КП «Чисте місто» Жовтоводської місько...,18.000,D1,NaN,D1221855500_00001_260716
3,4,Підметини та пух хутра штучного,1830.2.1.06,Підметини з цехів,ІV,0,35.200,40.5,38563155 ТОВ «Науково- дослідний інститут «УКР...,35.200,R5,NaN,D1221855500_00001_260716
4,5,Плівка чи оболонка на основі полімерів зіпсова...,1753.1.1.03,Відходи поліетилену,ІV,0,1.912,2.000,38563155 ТОВ «Науково- дослідний інститут «УКР...,1.912,R5,NaN,D1221855500_00001_260716
5,6,Макулатура паперова та картонна,7710.3.1.01,Відходи макулатури,ІV,0,3.120,6.202,38563155 ТОВ «Науково- дослідний інститут «УКР...,3.120,R5,NaN,D1221855500_00001_260716
6,7,Одяг зношений чи зіпсований,7710.3.1.13,Відходи спецодягу,ІV,0,0.104,0.195,38563155 ТОВ «Науково- дослідний інститут «УКР...,0.104,R5,NaN,D1221855500_00001_260716
7,1,Бій цегли,2640.2.9.02,Бій цегли,ІV,0,90.0,90.0,"26183742 філія ""Заставнівський райавтодор"", 59...",90.0,D1,NaN,D7321586001_00001_100616
8,2,Шлак паливний,4010.2.8.01,Шлак паливний,ІV,0,69.0,69.0,"26183742 філія ""Заставнівський райавтодор"", 59...",69.0,D1,NaN,D7321586001_00001_100616
9,1,Вичавки плодові та ягідні,1532.2.9.02,Вичавки плодові та ягідні,ІV,0,995.0,995.0,"36748453 КП ""Хотинблагоустрій"", Чернівецька об...",995.0,D1,NaN,D7325010100_00002_130716


In [ ]:
def column_scatter(dt):
    edrpou_f, name_f, address_f, fact_address_f, the_date_f = [], [], [], [], []
    for i in dt:
        edrpou = i.split()[0]
        if '.201' in i.split()[-1] or '.200' in i.split()[-1]:
            the_date = i.split()[-1]
            i = ' '.join(i.split()[1:-1])
        else:
            the_date = None
            i = ' '.join(i.split()[1:])
        name = i[:i.find(',')].strip()
        i = i[i.find(',') + 1:].strip()
        if 'Фактична адреса:' in i:
            address = i[:i.index('Фактична адреса:')].strip()
            fact_address = i[i.index('Фактична адреса:') + 16:].strip()
        else:
            address = i.strip()
            fact_address = None
        edrpou_f.append(edrpou)
        name_f.append(name)
        address_f.append(address)
        fact_address_f.append(fact_address)
        the_date_f.append(the_date)
#     print('edrpou', edrpou)
#     print('name', name)
#     print('adr', address)
#     print('fadr', fact_address)
#     print('date', the_date)
#     print('')
    return edrpou_f, name_f, address_f, fact_address_f, the_date_f

In [ ]:
data2 = pd.read_csv("./dec/data/fulls/Declarations_third_table.csv", sep=';')
data2['dest_edrpou'], data2['dest_name'], data2['dest_address'], data2['dest_fact_address'], data2['dest_date'] = column_scatter(data2['destination'])

In [4]:
pd.read_csv("dec/data/fulls/Declarations_third_splitted_table.csv", sep=';', dtype={'dest_edrpou': object})

,No,trash_name,code,alter_name,class,storage,value,value_prediction,destination,value_destination,operation,operation_desc,decl_num,dest_edrpou,dest_name,dest_address,data_fact_address,dest_date
0,1,"Масла та мастила моторні, трансмісійні нехлоро...",6000.2.8.09,Відпрацьовані мастила,ІІ,0,0.012,0.012,38563155 ТОВ «Науково- дослідний інститут «УКР...,0.012,R9,NaN,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27. ,",08.01.2014
1,2,"Матеріали обтиральні зіпсовані, відпрацьовані ...",7730.3.1.06,Промащене ганчір’я,ІІІ,0,0.100,0.211,38563155 ТОВ «Науково- дослідний інститут «УКР...,0.100,R5,NaN,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27. ,",08.01.2014
2,3,"Відходи комунальні (міські) змішані, у т. ч. с...",7720.3.1.01,Побутове сміття,ІV,0,18.000,84.18,38641065 КП «Чисте місто» Жовтоводської місько...,18.000,D1,NaN,D1221855500_00001_260716,38641065,"КП «Чисте місто» Жовтоводської міської ради""","52200,м. Жовті Води, вул. Петровського,56",NaN,NaN
3,4,Підметини та пух хутра штучного,1830.2.1.06,Підметини з цехів,ІV,0,35.200,40.5,38563155 ТОВ «Науково- дослідний інститут «УКР...,35.200,R5,NaN,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27.",NaN
4,5,Плівка чи оболонка на основі полімерів зіпсова...,1753.1.1.03,Відходи поліетилену,ІV,0,1.912,2.000,38563155 ТОВ «Науково- дослідний інститут «УКР...,1.912,R5,NaN,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27.",NaN
5,6,Макулатура паперова та картонна,7710.3.1.01,Відходи макулатури,ІV,0,3.120,6.202,38563155 ТОВ «Науково- дослідний інститут «УКР...,3.120,R5,NaN,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27.",NaN
6,7,Одяг зношений чи зіпсований,7710.3.1.13,Відходи спецодягу,ІV,0,0.104,0.195,38563155 ТОВ «Науково- дослідний інститут «УКР...,0.104,R5,NaN,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27.",NaN
7,1,Бій цегли,2640.2.9.02,Бій цегли,ІV,0,90.0,90.0,"26183742 філія ""Заставнівський райавтодор"", 59...",90.0,D1,NaN,D7321586001_00001_100616,26183742,"філія ""Заставнівський райавтодор""","59400, Чернівецька обл., м.Заставна, вул.Бажан...",NaN,NaN
8,2,Шлак паливний,4010.2.8.01,Шлак паливний,ІV,0,69.0,69.0,"26183742 філія ""Заставнівський райавтодор"", 59...",69.0,D1,NaN,D7321586001_00001_100616,26183742,"філія ""Заставнівський райавтодор""","59400, Чернівецька обл., м.Заставна, вул.Бажан...",NaN,NaN
9,1,Вичавки плодові та ягідні,1532.2.9.02,Вичавки плодові та ягідні,ІV,0,995.0,995.0,"36748453 КП ""Хотинблагоустрій"", Чернівецька об...",995.0,D1,NaN,D7325010100_00002_130716,36748453,"КП ""Хотинблагоустрій""","Чернівецька обл., м.Хотин, вул.Кобилянської, 15",NaN,NaN
